In [ ]:
%matplotlib notebook
%pylab

In [ ]:
%reload_ext autoreload
%autoreload 2

# Wave propagation algorithm for scalar advection

The one dimensional scalar advection equation considered here is 

\begin{equation}
q_t + u q_x = 0
\end{equation}

where $q(x,t)$ is a scalar tracer quantity and $u$ is constant velocity. 

In the wave propagation algorithm, the update formula is given in terms of *waves*, *speeds* and *fluctuations*.  

\begin{equation}
Q_i^{n+1} = Q_i^{n} - \frac{\Delta t}{\Delta x}\left(\mathcal A^+ \Delta Q_{i-1/2}  + \mathcal A^- \Delta Q_{i+1/2}\right) - \frac{\Delta t}{\Delta x}\left(\mathcal F_{i-1/2} + \mathcal F_{i+1/2}\right)
\end{equation}

where integer indices are values at cell centers, and half indices are used to indicate values at cell interfaces.  

For the scalar problem, the *fluctuations* $\mathcal A^{+} \Delta Q_{i-1/2}$ and $\mathcal A^{-} \Delta Q_{i+1/2}$ are defined as

\begin{eqnarray}
\mathcal A^{+} \Delta Q_{i-1/2} & \equiv & u^{+} (Q_{i} - Q_{i-1}) \\
\mathcal A^{-} \Delta Q_{i+1/2} & \equiv & u^{-} (Q_{i} - Q_{i-1}) \\
\end{eqnarray}

where

\begin{equation*}
u^+ \equiv \max(0,u) \qquad u^- \equiv \min(0,u)
\end{equation*}

The second order correction terms are given in terms of *waves* $\mathcal W_{i-1/2}$ and *speeds* $s_{i-1/2}$ at each interface as

\begin{equation*}
\mathcal F_{i-1/2} = \frac{1}{2}|s_{i-1/2}|\left(1 - \frac{\Delta t}{\Delta x} |s_{i-1/2}| \right) \mathcal W_{i-1/2}
\end{equation*}

For the scalar advection equation, waves defined as the *jump* in $Q$ at interface $x_{i-1/2}$. 

\begin{equation}
\mathcal W_{i-1/2} \equiv Q_{i} - Q_{i-1}
\end{equation}

Since $u$ is constant, the speeds are defined as $s_{i-1/2} \equiv u$.   To reduce oscillations in the solution, waves are typically limited with a *wave limiter*.  

\begin{equation}
\widetilde{\mathcal W}_{i-1/2} = \theta \; \mathcal W_{i-1/2}
\end{equation}

where $\theta$ is a coefficient that typically depends on neighboring $Q$ values $Q_{i-2},Q_{i-1},Q_{i},Q_{i+1},Q_{i+2}$.


## Riemann solver

The waves, speeds and fluctuations are computed in a *Riemann solver*.  The solver below
assumes that $Q_i$, $i = 0,1,...,N-1$ has been extended by a layer of two ghost cells at each end of the domain. 

\begin{equation}
Q_{ext} = \left[\bar{Q}_{-2}, \bar{Q}_{-1}, Q, \bar{Q}_{N}, \bar{Q}_{mx+1}\right]
\end{equation}

where $\bar{Q}_i$ are determined using either periodic boundary conditions, an exact solution, or some other condition, depending on the problem being solved.  Boundary conditions are 
supplied in the user function described below.  

The output are the waves $\mathcal W_{i-1/2}$, speeds $s_{i-1/2}$ and fluctations 
$\mathcal A^+ \Delta Q_{i-1/2}$ and $\mathcal A^- \Delta Q_{i-1/2}$ at cell interfaces
$i = -3/2, -1/2, ..., N-1/2, N+1/2$.  


In [ ]:
# Global data needed for Riemann solver and initialization routine

# Constant velocity
u = 1

def rp1_advection(Q_ext):
    """  Input : 
            Q_ext : Array of N+4 Q values.   Boundary conditions are included.
            
        Output : 
            waves  : Jump in Q at edges -3/2, -1/2, ..., N-1/2, N+1/2 (N+3 values total)
            speeds : Array of speeds (N+3 values)
            apdq   : Positive fluctuations (N+3 values)
            amdq   : Negative fluctuations (N+3 values)
        """
    
    # jump in Q at each interface
    waves = Q_ext[1:]-Q_ext[:-1]
    
    # speeds at each interface. 
    s = u*ones(waves.shape)
    
    # Fluctuations
    sm = where(s < 0,s, 0)
    sp = where(s > 0,s, 0)
    
    amdq = sm*waves
    apdq = sp*waves
    
    return waves,s,amdq,apdq



## Boundary conditions

The boundary conditions can be determined in a variety of ways.  Below, we prescribe
periodic boundary conditions. 

In [ ]:
def bc_periodic(Q):
    """ Extend Q with periodic boundary conditions """
    
    return concatenate((Q[-2:], Q, Q[:2]))

## Initial conditions

The solver should supply an initialization routine to initialize $q(x,t)$ at time $t=0$.

The exact solution is also supplied here, since it depends on the initial conditions.

In [ ]:
def qinit(x):
    #return exp(-80*(x-0.5)**2)
    return sin(2*pi*x)

# this relies on velocity u set above.
def qexact(x,t):
    # return qinit(mod(x-u*t,1))
    return qinit(x-u*t)

## Problem test

Below, we solve the scalar advection equation on the domain $[0,1]$ using periodic boundary conditions.  

In [ ]:
import wpa

# Spatial domain
ax = 0
bx = 1

# Temporal domain
Tfinal = 1

# Numerical parameters
cfl = 0.95

## Sample test and plot

In [ ]:

N = 128

# Compute maximum wave speed over the msh
umax = u    # Constant for scalar advection (set above in Riemann solver)
    
# Estimate time step and number of time steps to take
dx = (bx-ax)/N
dt_est = cfl*dx/umax;
M = int(floor(Tfinal/dt_est) + 1)
dt = Tfinal/M
        
Q,xc,tvec = wpa.claw1(ax,bx, N,  Tfinal, M, \
                     rp=rp1_advection, \
                     qinit=qinit, \
                     bc=bc_periodic, \
                     limiter_choice='MC')

## Plot the solution

In [ ]:
fig = figure(1)
clf()

q0 = Q[:,0]
hdl, = plot(xc,q0,'b.',markersize=5)

xfe = linspace(ax,bx,1000)
xfc = xfe[:-1] + dx/2
hdl_exact, = plot(xfc,qinit(xfc),'r-')

tstr = 'Wave Propagation : t = {:.4f}'
htitle = title(tstr.format(0),fontsize=18)

for i,t in enumerate(tvec):
    q = Q[:,i]
    
    hdl.set_ydata(q)

    qe = qinit(mod(xfc - u*t,1))
    hdl_exact.set_ydata(qe)
    
    xlabel('x',fontsize=16)
    ylabel('q(x,t)',fontsize=16)
    htitle.set_text(tstr.format(t))
    
    ylim([-1.1,1.1])

    fig.canvas.draw()        

## Convergence study

In [ ]:
Nv = [32,64,128,256,512,1024,2048]

error = empty((len(Nv),2))
print("{:>8s} {:>12s} {:>8s} {:>12s} {:>8s}".format('N','error(1)','rate(1)','error(2)','rate(2)'))
print("{:s}".format('-'*52))
for i,N in enumerate(Nv):

    # Compute maximum wave speed over the mesh
    umax = u    # Constant for scalar advection
    
    # Estimate time step
    dx = (bx-ax)/N
    dt_est = cfl*dx/umax;
    M = int(floor(Tfinal/dt_est) + 1)
    dt = Tfinal/M
        
    # First order solution
    Q1,xc,tvec = wpa.claw1(ax,bx, N,  Tfinal, M, \
                         rp=rp1_advection, \
                         qinit=qinit, \
                         bc=bc_periodic, \
                         limiter_choice=None,
                         second_order=False)

    # Second order solution
    Q2,xc,tvec = wpa.claw1(ax,bx, N,  Tfinal, M, \
                         rp=rp1_advection, \
                         qinit=qinit, \
                         bc=bc_periodic, \
                         limiter_choice=None,
                         second_order=True)
    
    qe = qexact(xc,Tfinal)    

    error[i,0] = abs(qe-Q1[:,-1]).max()  # First order error
    error[i,1] = abs(qe-Q2[:,-1]).max()  # Second order error

    rate_str = ['']*2
    if i == 0:
        rs = format("{:>8s}").format('---')
        rate_str = [rs]*2
    else:
        rs1 = log2(error[i-1,0]/error[i,0])  
        rs2 = log2(error[i-1,1]/error[i,1])
        rstr = "{:8.4f}"
        rate_str = [rstr.format(rs1), rstr.format(rs2)]

    print("{:8d} {:12.4e} {:s} {:12.4e} {:s}".format(N,error[i,0],rate_str[0], \
                                                    error[i,1],rate_str[1]))
    

In [ ]:
from cg_graphics import set_xticks

figure(2)
clf()

p = polyfit(log(Nv),log(error[:,0]),1)
loglog(Nv,error[:,0],'b.-',markersize=12,label="First order : slope = {:.2f}".format(p[0]))

p = polyfit(log(Nv),log(error[:,1]),1)
loglog(Nv,error[:,1],'r.-',markersize=12,label="Second order : slope = {:.2f}".format(p[0]))

set_xticks(Nv)
xlabel('N',fontsize=16)
ylabel('Error',fontsize=16)
title('WPA : Convergence')
       
legend()

show()